<h1><center>CS457 Lab 4 <br />TCP Generic Application Client and Server</center></h1>

### Introduction

This lab finalizes our set of TCP tutorials by implementing a generic asynchronous multi-connection application server that handles application data encoding/decoding as well as better error handling.  

In the first part of this lab you will examine and execute an existing client/server application to understand how its underlying class structure works.  Then, after reading a lengthy but informative explanation, you will make a small modification to the python code to implement two simple new commands: 

* `double`: takes an integer and returns the integer value multiplied by two
* `negate`: takes an integer and returns the negative value of that integer

Sounds simple?  Not unless you truly understand how data is formatted "on the wire"...

#### Yes, I know this lab is long and there is a lot to read

But you need to understand the concepts introduced in this lab.

#### Acknowledgements and Citations

This lab is sourced with permission from the tutorial by Nathan Jennings [Socket Programming in Python](https://realpython.com/python-sockets/).  It has been edited for clarity and slightly modified and tuned for CS457.  The tutorial has also been modified to run within a Jupyter Notebook.  

------

### Test Drive the Application

Before we get into the structure and inner workings of the application, you should first simply run it to see what it does.  In essence, the client creates a packet containing a command and associated parameter to the server.  The server executes the command and returns the results to the client.  The only built-in command is "search", which will return information for only a few items that have been hard-coded into the application.  Any other command besides "search" will put the server in binary mode and it will simply return the first 10 bytes of the command and parameters.  Here are the steps for you to run the program:

1. Execute the four code blocks in the cells below to save the files into your current working directory.
2. Per the instructions below, create a terminal window, navigate to the working directory, and start the server.  The server takes two command line parameters, an IP address (e.g. '127.0.0.1' or '' for any IP) and a port number.
3. Then, per the instructions below, create another terminal window, navigate to the working directory, and run several experiments by invoking the client program with different parameters.  The client takes four command line parameters: an IP address and port number, a command (e.g. search), and a command parameter indicating what to search for.

In [1]:
%%writefile app-client.py

#!/usr/bin/env python3

import sys
import socket
import selectors
import traceback

import libclient

sel = selectors.DefaultSelector()


def create_request(action, value):
    if action == "search":
        return dict(
            type="text/json",
            encoding="utf-8",
            content=dict(action=action, value=value),
        )
    else:
        return dict(
            type="binary/custom-client-binary-type",
            encoding="binary",
            content=bytes(action + value, encoding="utf-8"),
        )


def start_connection(host, port, request):
    addr = (host, port)
    print("starting connection to", addr)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setblocking(False)
    sock.connect_ex(addr)
    events = selectors.EVENT_READ | selectors.EVENT_WRITE
    message = libclient.Message(sel, sock, addr, request)
    sel.register(sock, events, data=message)


if len(sys.argv) != 5:
    print("usage:", sys.argv[0], "<host> <port> <action> <value>")
    sys.exit(1)

host, port = sys.argv[1], int(sys.argv[2])
action, value = sys.argv[3], sys.argv[4]
request = create_request(action, value)
start_connection(host, port, request)

try:
    while True:
        events = sel.select(timeout=1)
        for key, mask in events:
            message = key.data
            try:
                message.process_events(mask)
            except Exception:
                print(
                    "main: error: exception for",
                    f"{message.addr}:\n{traceback.format_exc()}",
                )
                message.close()
        # Check for a socket being monitored to continue.
        if not sel.get_map():
            break
except KeyboardInterrupt:
    print("caught keyboard interrupt, exiting")
finally:
    sel.close()

Writing app-client.py


In [2]:
%%writefile app-server.py

#!/usr/bin/env python3

import sys
import socket
import selectors
import traceback

import libserver

sel = selectors.DefaultSelector()


def accept_wrapper(sock):
    conn, addr = sock.accept()  # Should be ready to read
    print("accepted connection from", addr)
    conn.setblocking(False)
    message = libserver.Message(sel, conn, addr)
    sel.register(conn, selectors.EVENT_READ, data=message)


if len(sys.argv) != 3:
    print("usage:", sys.argv[0], "<host> <port>")
    sys.exit(1)

host, port = sys.argv[1], int(sys.argv[2])
lsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Avoid bind() exception: OSError: [Errno 48] Address already in use
lsock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
lsock.bind((host, port))
lsock.listen()
print("listening on", (host, port))
lsock.setblocking(False)
sel.register(lsock, selectors.EVENT_READ, data=None)

try:
    while True:
        events = sel.select(timeout=None)
        for key, mask in events:
            if key.data is None:
                accept_wrapper(key.fileobj)
            else:
                message = key.data
                try:
                    message.process_events(mask)
                except Exception:
                    print(
                        "main: error: exception for",
                        f"{message.addr}:\n{traceback.format_exc()}",
                    )
                    message.close()
except KeyboardInterrupt:
    print("caught keyboard interrupt, exiting")
finally:
    sel.close()

Writing app-server.py


In [3]:
%%writefile libclient.py

import sys
import selectors
import json
import io
import struct


class Message:
    def __init__(self, selector, sock, addr, request):
        self.selector = selector
        self.sock = sock
        self.addr = addr
        self.request = request
        self._recv_buffer = b""
        self._send_buffer = b""
        self._request_queued = False
        self._jsonheader_len = None
        self.jsonheader = None
        self.response = None

    def _set_selector_events_mask(self, mode):
        """Set selector to listen for events: mode is 'r', 'w', or 'rw'."""
        if mode == "r":
            events = selectors.EVENT_READ
        elif mode == "w":
            events = selectors.EVENT_WRITE
        elif mode == "rw":
            events = selectors.EVENT_READ | selectors.EVENT_WRITE
        else:
            raise ValueError(f"Invalid events mask mode {repr(mode)}.")
        self.selector.modify(self.sock, events, data=self)

    def _read(self):
        try:
            # Should be ready to read
            data = self.sock.recv(4096)
        except BlockingIOError:
            # Resource temporarily unavailable (errno EWOULDBLOCK)
            pass
        else:
            if data:
                self._recv_buffer += data
            else:
                raise RuntimeError("Peer closed.")

    def _write(self):
        if self._send_buffer:
            print("sending", repr(self._send_buffer), "to", self.addr)
            try:
                # Should be ready to write
                sent = self.sock.send(self._send_buffer)
            except BlockingIOError:
                # Resource temporarily unavailable (errno EWOULDBLOCK)
                pass
            else:
                self._send_buffer = self._send_buffer[sent:]

    def _json_encode(self, obj, encoding):
        return json.dumps(obj, ensure_ascii=False).encode(encoding)

    def _json_decode(self, json_bytes, encoding):
        tiow = io.TextIOWrapper(
            io.BytesIO(json_bytes), encoding=encoding, newline=""
        )
        obj = json.load(tiow)
        tiow.close()
        return obj

    def _create_message(
        self, *, content_bytes, content_type, content_encoding
    ):
        jsonheader = {
            "byteorder": sys.byteorder,
            "content-type": content_type,
            "content-encoding": content_encoding,
            "content-length": len(content_bytes),
        }
        jsonheader_bytes = self._json_encode(jsonheader, "utf-8")
        message_hdr = struct.pack(">H", len(jsonheader_bytes))
        message = message_hdr + jsonheader_bytes + content_bytes
        return message

    def _process_response_json_content(self):
        content = self.response
        result = content.get("result")
        print(f"got result: {result}")

    def _process_response_binary_content(self):
        content = self.response
        print(f"got response: {repr(content)}")

    def process_events(self, mask):
        if mask & selectors.EVENT_READ:
            self.read()
        if mask & selectors.EVENT_WRITE:
            self.write()

    def read(self):
        self._read()

        if self._jsonheader_len is None:
            self.process_protoheader()

        if self._jsonheader_len is not None:
            if self.jsonheader is None:
                self.process_jsonheader()

        if self.jsonheader:
            if self.response is None:
                self.process_response()

    def write(self):
        if not self._request_queued:
            self.queue_request()

        self._write()

        if self._request_queued:
            if not self._send_buffer:
                # Set selector to listen for read events, we're done writing.
                self._set_selector_events_mask("r")

    def close(self):
        print("closing connection to", self.addr)
        try:
            self.selector.unregister(self.sock)
        except Exception as e:
            print(
                f"error: selector.unregister() exception for",
                f"{self.addr}: {repr(e)}",
            )

        try:
            self.sock.close()
        except OSError as e:
            print(
                f"error: socket.close() exception for",
                f"{self.addr}: {repr(e)}",
            )
        finally:
            # Delete reference to socket object for garbage collection
            self.sock = None

    def queue_request(self):
        content = self.request["content"]
        content_type = self.request["type"]
        content_encoding = self.request["encoding"]
        if content_type == "text/json":
            req = {
                "content_bytes": self._json_encode(content, content_encoding),
                "content_type": content_type,
                "content_encoding": content_encoding,
            }
        else:
            req = {
                "content_bytes": content,
                "content_type": content_type,
                "content_encoding": content_encoding,
            }
        message = self._create_message(**req)
        self._send_buffer += message
        self._request_queued = True

    def process_protoheader(self):
        hdrlen = 2
        if len(self._recv_buffer) >= hdrlen:
            self._jsonheader_len = struct.unpack(
                ">H", self._recv_buffer[:hdrlen]
            )[0]
            self._recv_buffer = self._recv_buffer[hdrlen:]

    def process_jsonheader(self):
        hdrlen = self._jsonheader_len
        if len(self._recv_buffer) >= hdrlen:
            self.jsonheader = self._json_decode(
                self._recv_buffer[:hdrlen], "utf-8"
            )
            self._recv_buffer = self._recv_buffer[hdrlen:]
            for reqhdr in (
                "byteorder",
                "content-length",
                "content-type",
                "content-encoding",
            ):
                if reqhdr not in self.jsonheader:
                    raise ValueError(f'Missing required header "{reqhdr}".')

    def process_response(self):
        content_len = self.jsonheader["content-length"]
        if not len(self._recv_buffer) >= content_len:
            return
        data = self._recv_buffer[:content_len]
        self._recv_buffer = self._recv_buffer[content_len:]
        if self.jsonheader["content-type"] == "text/json":
            encoding = self.jsonheader["content-encoding"]
            self.response = self._json_decode(data, encoding)
            print("received response", repr(self.response), "from", self.addr)
            self._process_response_json_content()
        else:
            # Binary or unknown content-type
            self.response = data
            print(
                f'received {self.jsonheader["content-type"]} response from',
                self.addr,
            )
            self._process_response_binary_content()
        # Close when response has been processed
        self.close()

Writing libclient.py


In [4]:
%%writefile libserver.py

import sys
import selectors
import json
import io
import struct

request_search = {
    "morpheus": "Follow the white rabbit. \U0001f430",
    "ring": "In the caves beneath the Misty Mountains. \U0001f48d",
    "\U0001f436": "\U0001f43e Playing ball! \U0001f3d0",
}


class Message:
    def __init__(self, selector, sock, addr):
        self.selector = selector
        self.sock = sock
        self.addr = addr
        self._recv_buffer = b""
        self._send_buffer = b""
        self._jsonheader_len = None
        self.jsonheader = None
        self.request = None
        self.response_created = False

    def _set_selector_events_mask(self, mode):
        """Set selector to listen for events: mode is 'r', 'w', or 'rw'."""
        if mode == "r":
            events = selectors.EVENT_READ
        elif mode == "w":
            events = selectors.EVENT_WRITE
        elif mode == "rw":
            events = selectors.EVENT_READ | selectors.EVENT_WRITE
        else:
            raise ValueError(f"Invalid events mask mode {repr(mode)}.")
        self.selector.modify(self.sock, events, data=self)

    def _read(self):
        try:
            # Should be ready to read
            data = self.sock.recv(4096)
        except BlockingIOError:
            # Resource temporarily unavailable (errno EWOULDBLOCK)
            pass
        else:
            if data:
                self._recv_buffer += data
            else:
                raise RuntimeError("Peer closed.")

    def _write(self):
        if self._send_buffer:
            print("sending", repr(self._send_buffer), "to", self.addr)
            try:
                # Should be ready to write
                sent = self.sock.send(self._send_buffer)
            except BlockingIOError:
                # Resource temporarily unavailable (errno EWOULDBLOCK)
                pass
            else:
                self._send_buffer = self._send_buffer[sent:]
                # Close when the buffer is drained. The response has been sent.
                if sent and not self._send_buffer:
                    self.close()

    def _json_encode(self, obj, encoding):
        return json.dumps(obj, ensure_ascii=False).encode(encoding)

    def _json_decode(self, json_bytes, encoding):
        tiow = io.TextIOWrapper(
            io.BytesIO(json_bytes), encoding=encoding, newline=""
        )
        obj = json.load(tiow)
        tiow.close()
        return obj

    def _create_message(
        self, *, content_bytes, content_type, content_encoding
    ):
        jsonheader = {
            "byteorder": sys.byteorder,
            "content-type": content_type,
            "content-encoding": content_encoding,
            "content-length": len(content_bytes),
        }
        jsonheader_bytes = self._json_encode(jsonheader, "utf-8")
        message_hdr = struct.pack(">H", len(jsonheader_bytes))
        message = message_hdr + jsonheader_bytes + content_bytes
        return message

    def _create_response_json_content(self):
        action = self.request.get("action")
        if action == "search":
            query = self.request.get("value")
            answer = request_search.get(query) or f'No match for "{query}".'
            content = {"result": answer}
        else:
            content = {"result": f'Error: invalid action "{action}".'}
        content_encoding = "utf-8"
        response = {
            "content_bytes": self._json_encode(content, content_encoding),
            "content_type": "text/json",
            "content_encoding": content_encoding,
        }
        return response

    def _create_response_binary_content(self):
        response = {
            "content_bytes": b"First 10 bytes of request: "
            + self.request[:10],
            "content_type": "binary/custom-server-binary-type",
            "content_encoding": "binary",
        }
        return response

    def process_events(self, mask):
        if mask & selectors.EVENT_READ:
            self.read()
        if mask & selectors.EVENT_WRITE:
            self.write()

    def read(self):
        self._read()

        if self._jsonheader_len is None:
            self.process_protoheader()

        if self._jsonheader_len is not None:
            if self.jsonheader is None:
                self.process_jsonheader()

        if self.jsonheader:
            if self.request is None:
                self.process_request()

    def write(self):
        if self.request:
            if not self.response_created:
                self.create_response()

        self._write()

    def close(self):
        print("closing connection to", self.addr)
        try:
            self.selector.unregister(self.sock)
        except Exception as e:
            print(
                f"error: selector.unregister() exception for",
                f"{self.addr}: {repr(e)}",
            )

        try:
            self.sock.close()
        except OSError as e:
            print(
                f"error: socket.close() exception for",
                f"{self.addr}: {repr(e)}",
            )
        finally:
            # Delete reference to socket object for garbage collection
            self.sock = None

    def process_protoheader(self):
        hdrlen = 2
        if len(self._recv_buffer) >= hdrlen:
            self._jsonheader_len = struct.unpack(
                ">H", self._recv_buffer[:hdrlen]
            )[0]
            self._recv_buffer = self._recv_buffer[hdrlen:]

    def process_jsonheader(self):
        hdrlen = self._jsonheader_len
        if len(self._recv_buffer) >= hdrlen:
            self.jsonheader = self._json_decode(
                self._recv_buffer[:hdrlen], "utf-8"
            )
            self._recv_buffer = self._recv_buffer[hdrlen:]
            for reqhdr in (
                "byteorder",
                "content-length",
                "content-type",
                "content-encoding",
            ):
                if reqhdr not in self.jsonheader:
                    raise ValueError(f'Missing required header "{reqhdr}".')

    def process_request(self):
        content_len = self.jsonheader["content-length"]
        if not len(self._recv_buffer) >= content_len:
            return
        data = self._recv_buffer[:content_len]
        self._recv_buffer = self._recv_buffer[content_len:]
        if self.jsonheader["content-type"] == "text/json":
            encoding = self.jsonheader["content-encoding"]
            self.request = self._json_decode(data, encoding)
            print("received request", repr(self.request), "from", self.addr)
        else:
            # Binary or unknown content-type
            self.request = data
            print(
                f'received {self.jsonheader["content-type"]} request from',
                self.addr,
            )
        # Set selector to listen for write events, we're done reading.
        self._set_selector_events_mask("w")

    def create_response(self):
        if self.jsonheader["content-type"] == "text/json":
            response = self._create_response_json_content()
        else:
            # Binary or unknown content-type
            response = self._create_response_binary_content()
        message = self._create_message(**response)
        self.response_created = True
        self._send_buffer += message

Writing libserver.py


----

### Running the Application Client and Server

In these examples, I’ll run the server so it listens on all interfaces by passing an empty string for the host argument. This will allow me to run the client and connect from a virtual machine that could be on the same machine or somewhere else in the network.

At this stage, don't worry if you don't understand what is happening or how to interpret the output.  Just make sure you get output similar to the ones displayed in this cell.  We will go into the gory details shortly.

First, let’s start the server:
>```
$ ./app-server.py '' 65432
listening on ('', 65432)
```

Now let’s run the client and enter a search. Let’s see if we can find the mysterious "morpheus" (apologies to lovers of the Matrix):

>```
$ ./app-client.py 10.0.1.1 65432 search morpheus
starting connection to ('10.0.1.1', 65432)
sending b'\x00d{"byteorder": "big", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 41}{"action": "search", "value": "morpheus"}' to ('10.0.1.1', 65432)
received response {'result': 'Follow the white rabbit. 🐰'} from ('10.0.1.1', 65432)
got result: Follow the white rabbit. 🐰
closing connection to ('10.0.1.1', 65432)
```

My terminal is running a shell that’s using a text encoding of Unicode (UTF-8), so the output above prints nicely with emojis.

Let’s see if we can find the puppies (you will probably have to copy and paste the puppy emoji into the command line):

>```
$ ./app-client.py 10.0.1.1 65432 search 🐶
starting connection to ('10.0.1.1', 65432)
sending b'\x00d{"byteorder": "big", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 37}{"action": "search", "value": "\xf0\x9f\x90\xb6"}' to ('10.0.1.1', 65432)
received response {'result': '🐾 Playing ball! 🏐'} from ('10.0.1.1', 65432)
got result: 🐾 Playing ball! 🏐
closing connection to ('10.0.1.1', 65432)
```

Notice the byte string sent over the network for the request in the sending line. It’s easier to see if you look for the bytes printed in hex that represent the puppy emoji: \xf0\x9f\x90\xb6. I was able to enter the emoji for the search since my terminal is using Unicode with the encoding UTF-8.

This demonstrates that we’re sending raw bytes over the network and they need to be decoded by the receiver to be interpreted correctly. This is why we went to all of the trouble to create a header that contains the content type and encoding.

Here’s the server output from both client connections above:

>```
accepted connection from ('10.0.2.2', 55340)
received request {'action': 'search', 'value': 'morpheus'} from ('10.0.2.2', 55340)
sending b'\x00g{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 43}{"result": "Follow the white rabbit. \xf0\x9f\x90\xb0"}' to ('10.0.2.2', 55340)
closing connection to ('10.0.2.2', 55340)
```
 
>```
accepted connection from ('10.0.2.2', 55338)
received request {'action': 'search', 'value': '🐶'} from ('10.0.2.2', 55338)
sending b'\x00g{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 37}{"result": "\xf0\x9f\x90\xbe Playing ball! \xf0\x9f\x8f\x90"}' to ('10.0.2.2', 55338)
closing connection to ('10.0.2.2', 55338)
```

Look at the sending line to see the bytes that were written to the client’s socket. This is the server’s response message.

You can also test sending binary requests to the server if the action argument is anything other than search:
>```
$ ./app-client.py 10.0.1.1 65432 binary 😃
starting connection to ('10.0.1.1', 65432)
sending b'\x00|{"byteorder": "big", "content-type": "binary/custom-client-binary-type", "content-encoding": "binary", "content-length": 10}binary\xf0\x9f\x98\x83' to ('10.0.1.1', 65432)
received binary/custom-server-binary-type response from ('10.0.1.1', 65432)
got response: b'First 10 bytes of request: binary\xf0\x9f\x98\x83'
closing connection to ('10.0.1.1', 65432)
```


Since the request’s content-type is not text/json, the server treats it as a custom binary type and doesn’t perform JSON decoding. It simply prints the content-type and returns the first 10 bytes to the client:

>```
$ ./app-server.py '' 65432
listening on ('', 65432)
accepted connection from ('10.0.2.2', 55320)
received binary/custom-client-binary-type request from ('10.0.2.2', 55320)
sending b'\x00\x7f{"byteorder": "little", "content-type": "binary/custom-server-binary-type", "content-encoding": "binary", "content-length": 37}First 10 bytes of request: binary\xf0\x9f\x98\x83' to ('10.0.2.2', 55320)
closing connection to ('10.0.2.2', 55320)
```


---

### Application Overview -- How does this  work?

The multi-connection client and server example you explored in CS457 Lab 3 is definitely an improvement compared with where we started. However, let’s take one more step and address the shortcomings of the previous “multiconn” example in our final TCP implementation: a generic application client and server.

We want a client and server that handles errors appropriately so that other connections aren’t affected. Obviously, our client or server shouldn’t come crashing down in a ball of fury if an exception isn’t caught. This is something we haven’t discussed up until now and has been intentionally left out for brevity and clarity in the examples.  We also want to show how to handle data formatting and data interpretation in our generic application server.

At first, these additions to the code might appear simple to implement -- but beware!!!  Without thinking ahead, you will end up with a mess of spaghetti code that will be buggy, unreadable, and unmaintainable.  This client/server example makes use of a CLASS structure that maintains state and makes the code much easier to navigate.  

#### Error Handling

Now that you’re familiar with basic API, non-blocking sockets, and `select()`, we can add some error handling and discuss a custom class designed to handle data formats.

First, let’s address the errors:

>“All errors raise exceptions. The normal exceptions for invalid argument types and out-of-memory conditions can be raised; starting from Python 3.3, errors related to socket or address semantics raise OSError or one of its subclasses.” [(Source)](https://docs.python.org/3/library/socket.html)

We need to catch `OSError`. Another thing that hasn't been mentioned in relation to errors is timeouts. You’ll see them discussed in many places in the documentation. Timeouts happen and are a “normal” error. Hosts and routers are rebooted, switch ports go bad, cables go bad, cables get unplugged, you name it. You should be prepared for these and other errors and handle them in your code.

#### Handling TCP data streams:  buffering

As hinted by the socket type `socket.SOCK_STREAM`, when using TCP, you’re reading from a continuous stream of bytes. It’s like reading from a file on disk, but instead you’re reading bytes from the network.

However, unlike reading a file, there’s no `f.seek()`. In other words, you can’t reposition the socket pointer, if there was one, and move randomly around the data reading whatever, whenever you’d like.

When bytes arrive at your socket, there are network buffers involved. Once you’ve read them, they need to be saved somewhere. Calling `recv()` again reads the next stream of bytes available from the socket.

What this means is that you’ll be reading from the socket in chunks. You need to call `recv()` and save the data in a buffer until you’ve read enough bytes to have a complete message that makes sense to your application.

It’s up to you to define and keep track of where the message boundaries are. As far as the TCP socket is concerned, it’s just sending and receiving raw bytes to and from the network. It knows nothing about what those raw bytes mean.

#### Handling TCP data streams:   data encoding/decoding

This bring us to defining an application-layer protocol. What’s an application-layer protocol? Put simply, your application will send and receive messages. These messages are your application’s protocol.

In other words, the length and format you choose for these messages define the semantics and behavior of your application. This is directly related to what was explained in the previous paragraph regarding reading bytes from the socket. When you’re reading bytes with `recv()`, you need to keep up with how many bytes were read and figure out where the message boundaries are.

How is this done? One way is to always send fixed-length messages. If they’re always the same size, then it’s easy. When you’ve read that number of bytes into a buffer, then you know you have one complete message.

However, using fixed-length messages is inefficient for small messages where you’d need to use padding to fill them out. Also, you’re still left with the problem of what to do about data that doesn’t fit into one message.

In this tutorial, we’ll take a generic approach. An approach that’s used by many protocols, including HTTP. We’ll prefix messages with a header that includes the content length as well as any other fields we need. By doing this, we’ll only need to keep up with the header. Once we’ve read the header, we can process it to determine the length of the message’s content and then read that number of bytes to consume it.

We’ll implement this by creating a **custom class** that can send and receive messages that contain text or binary data. You can improve and extend it for your own applications. The most important thing is that you’ll be able to see an example of how this is done.

*The next issue is extremely important*. As we talked about earlier, when sending and receiving data via sockets, you’re sending and receiving raw bytes.

If you receive data and want to use it in a context where it’s interpreted as multiple bytes, for example a 4-byte integer, you **must**  take into account the differences in byte ordering between the host and the network.  The network will send the data in *network byte order*, a format that might be different than the byte order native to your machine’s CPU. Furthermore, the client or server on the other end of the network connection could have a CPU that uses a different byte order than your own.  We will give a better definition of "*network byte-order*" vs "*host byte-order*" and give examples on how to do byte-order conversion a little bit later.

#### What is your endianness?

Byte order is referred to as a CPU’s [endianness](https://en.wikipedia.org/wiki/Endianness). See Byte Endianness in the reference section for details.

You can easily determine the byte order of your machine by using `sys.byteorder`. Give it a try by executing the following cell:

In [5]:
import sys
sys.byteorder

'little'

  We’ll avoid the endianness issue when we define our **message header** by taking advantage of Unicode using the encoding UTF-8. Since UTF-8 uses an 8-bit encoding, there are no byte ordering issues.  You can find an explanation in Python’s Encodings and Unicode documentation.  

The **message payload**, on the other hand, could have any type of data: short integers, floating point, binary data, etc.   We’ll use an explicit type and encoding specified in the header so the application can interpret the content that’s being sent in the message payload. This will allow us to transfer any data we’d like (text or binary), in any format.

#### Network Order vs Host Order, and packing binary structures

The byte ordering used in TCP/IP is big-endian and is referred to as network order. Network order is used to represent integers in lower layers of the protocol stack, like IP addresses and port numbers. Python’s socket module includes functions that convert integers to and from network and host byte order:

|Function|	Description|
|:----|:----|
|`socket.ntohl(x)`|	Convert 32-bit positive integers from network to host byte order. On machines where the host byte order is the same as network byte order, this is a no-op; otherwise, it performs a 4-byte swap operation.|
|`socket.ntohs(x)`|	Convert 16-bit positive integers from network to host byte order. On machines where the host byte order is the same as network byte order, this is a no-op; otherwise, it performs a 2-byte swap operation.|
|`socket.htonl(x)`|	Convert 32-bit positive integers from host to network byte order. On machines where the host byte order is the same as network byte order, this is a no-op; otherwise, it performs a 4-byte swap operation.|
|`socket.htons(x)`|	Convert 16-bit positive integers from host to network byte order. On machines where the host byte order is the same as network byte order, this is a no-op; otherwise, it performs a 2-byte swap operation.|

You can also use the [struct module](https://docs.python.org/3/library/struct.html) to pack and unpack binary data using format strings:

>```
import struct
network_byteorder_int = struct.pack('>H', 256)
python_int = struct.unpack('>H', network_byteorder_int)[0]
```

-----

### Application Client and Server

#### Application Protocol Header

Let’s fully define the protocol header. The protocol header is:

* Variable-length text
* Unicode with the encoding UTF-8
* A Python dictionary serialized using [JSON](https://realpython.com/python-json/)

The required headers, or sub-headers, in the protocol header’s dictionary are as follows:


|Name | Description |
|:----|:----|
|byteorder | The byte order of the machine (uses sys.byteorder). This may not be required for your application.|
|content-length | The length of the content in bytes. |
|content-type | The type of content in the payload, for example, text/json or binary/my-binary-type. |
| content-encoding | The encoding used by the content, for example, utf-8 for Unicode text or binary for binary data.
These headers inform the receiver about the content in the payload of the message. This allows you to send arbitrary data while providing enough information so the content can be decoded and interpreted correctly by the receiver. Since the headers are in a dictionary, it’s easy to add additional headers by inserting key/value pairs as needed. |


#### Sending an Application Message
There’s still a bit of a problem. We have a variable-length header, which is nice and flexible, but how do you know the length of the header when reading it with recv()?

When we previously talked about using recv() and message boundaries, I mentioned that fixed-length headers can be inefficient. That’s true, but we’re going to use a small, 2-byte, fixed-length header to prefix the JSON header that contains its length.

You can think of this as a hybrid approach to sending messages. In effect, we’re bootstrapping the message receive process by sending the length of the header first. This makes it easy for our receiver to deconstruct the message.

To give you a better idea of the message format, let’s look at a message in its entirety:

<img src="http://www.cs.colostate.edu/~cs457/images/jupyter/genericAppHeader.jpg" width="500px" alt="header format" />

A message starts with a fixed-length header of 2 bytes that’s an integer in network byte order. This is the length of the next header, the variable-length JSON header. Once we’ve read 2 bytes with recv(), then we know we can process the 2 bytes as an integer and then read that number of bytes before decoding the UTF-8 JSON header.

The JSON header contains a dictionary of additional headers. One of those is content-length, which is the number of bytes of the message’s content (not including the JSON header). Once we’ve called recv() and read content-length bytes, we’ve reached a message boundary and read an entire message.

#### Application Message Class

Finally, the payoff! Let’s look at the Message class and see how it’s used with select() when read and write events happen on the socket.

For this example application, I had to come up with an idea for what types of messages the client and server would use. We’re far beyond toy echo clients and servers at this point.

To keep things simple and still demonstrate how things would work in a real application, I created an application protocol that implements a basic search feature. The client sends a search request and the server does a lookup for a match. If the request sent by the client isn’t recognized as a search, the server assumes it’s a binary request and returns a binary response.

After reading the following sections, running the examples, and experimenting with the code, you’ll see how things work. You can then use the Message class as a starting point and modify it for your own use.

We’re really not that far off from the “multiconn” client and server example. The event loop code stays the same in app-client.py and app-server.py. What I’ve done is move the message code into a class named Message and added methods to support reading, writing, and processing of the headers and content. This is a great example for using a class.

As we discussed before and you’ll see below, working with sockets involves keeping state. By using a class, we keep all of the state, data, and code bundled together in an organized unit. An instance of the class is created for each socket in the client and server when a connection is started or accepted.

The class is mostly the same for both the client and the server for the wrapper and utility methods. They start with an underscore, like Message._json_encode(). These methods simplify working with the class. They help other methods by allowing them to stay shorter and support the DRY principle.

The server’s Message class works in essentially the same way as the client’s and vice-versa. The difference being that the client initiates the connection and sends a request message, followed by processing the server’s response message. Conversely, the server waits for a connection, processes the client’s request message, and then sends a response message.

It looks like this:

|Step |	Endpoint	| Action / Message Content |
|:---|:---|:---|
|1 |	Client	|Sends a Message containing request content|
|2	|Server	|Receives and processes client request Message|
|3	|Server|	Sends a Message containing response content|
|4	|Client|	Receives and processes server response Message|

Here’s the file and code layout:

|Application	|File	|Code|
|:---|:---|:---|
|Server	|app-server.py	|The server’s main script|
|Server	|libserver.py	|The server’s Message class|
|Client	|app-client.py	|The client’s main script|
|Client	|libclient.py	|The client’s Message class|


#### Message Entry Point

I’d like to discuss how the Message class works by first mentioning an aspect of its design that wasn’t immediately obvious to me. Only after refactoring it at least five times did I arrive at what it is currently. Why? Managing state.

After a Message object is created, it’s associated with a socket that’s monitored for events using `selector.register()`:


>```
message = libserver.Message(sel, conn, addr)
sel.register(conn, selectors.EVENT_READ, data=message)
```

>Note: Some of the code examples in this section are from the server’s main script and Message class, but this section and discussion applies equally to the client as well. I’ll show and explain the client’s version when it differs.

When events are ready on the socket, they’re returned by selector.select(). We can then get a reference back to the message object using the data attribute on the key object and call a method in `Message`:

>```
while True:
    events = sel.select(timeout=None)
    for key, mask in events:
        # ...
        message = key.data
        message.process_events(mask)
        ```
        
Looking at the event loop above, you’ll see that `sel.select()` is in the driver’s seat. It’s blocking, waiting at the top of the loop for events. It’s responsible for waking up when read and write events are ready to be processed on the socket. Which means, indirectly, it’s also responsible for calling the method `process_events()`. This is what I mean when I say the method `process_events()` is the entry point.

Let’s see what the `process_events()` method does:

>```
def process_events(self, mask):
    if mask & selectors.EVENT_READ:
        self.read()
    if mask & selectors.EVENT_WRITE:
        self.write()
      ```
      
That’s good: `process_events()` is simple. It can only do two things: call `read()` and `write()`.

This brings us back to managing state. After a few refactorings, I decided that if another method depended on state variables having a certain value, then they would only be called from `read()` and `write()`. This keeps the logic as simple as possible as events come in on the socket for processing.

This may seem obvious, but the first few iterations of the class were a mix of some methods that checked the current state and, depending on their value, called other methods to process data outside `read()` or `write()`. In the end, this proved too complex to manage and keep up with.

You should definitely modify the class to suit your own needs so it works best for you, but I’d recommend that you keep the state checks and the calls to methods that depend on that state to the `read()` and `write()` methods if possible.

Let’s look at `read()`. This is the server’s version, but the client’s is the same. It just uses a different method name, `process_response()` instead of `process_request()`:

>```
def read(self):
    self._read()
```
>```
    if self._jsonheader_len is None:
        self.process_protoheader()
```
>```
    if self._jsonheader_len is not None:
        if self.jsonheader is None:
            self.process_jsonheader()
```
>```
    if self.jsonheader:
        if self.request is None:
            self.process_request()
```
            
The `_read()` method is called first. It calls `socket.recv()` to read data from the socket and store it in a receive buffer.

Remember that when `socket.recv()` is called, all of the data that makes up a complete message may not have arrived yet. `socket.recv()` may need to be called again. This is why there are state checks for each part of the message before calling the appropriate method to process it.

Before a method processes its part of the message, it first checks to make sure enough bytes have been read into the receive buffer. If there are, it processes its respective bytes, removes them from the buffer and writes its output to a variable that’s used by the next processing stage. Since there are three components to a message, there are three state checks and process method calls:

|Message Component |	Method	|Output|
|:---|:---|:---|
|Fixed-length header	|`process_protoheader()`	|`self._jsonheader_len`|
|JSON header	|`process_jsonheader()`	|`self.jsonheader`|
|Content	|`process_request()`	|`self.request`|

Next, let’s look at `write()`. This is the server’s version:

>```
def write(self):
    if self.request:
        if not self.response_created:
            self.create_response()
```
>```
    self._write()
```

`write()` checks first for a request. If one exists and a response hasn’t been created, `create_response()` is called. `create_response()` sets the state variable `response_created` and writes the response to the send buffer.

The `_write()` method calls `socket.send()` if there’s data in the send buffer.

Remember that when `socket.send()` is called, all of the data in the send buffer may not have been queued for transmission. The network buffers for the socket may be full, and `socket.send()` may need to be called again. This is why there are state checks. `create_response()` should only be called once, but it’s expected that `_write()` will need to be called multiple times.

The client version of `write()` is similar:

>```
def write(self):
    if not self._request_queued:
        self.queue_request()
```
>```
    self._write()
```
>```
    if self._request_queued:
        if not self._send_buffer:
            # Set selector to listen for read events, we're done writing.
            self._set_selector_events_mask('r')
```

Since the client initiates a connection to the server and sends a request first, the state variable `_request_queued` is checked. If a request hasn’t been queued, it calls `queue_request(`). `queue_request()` creates the request and writes it to the send buffer. It also sets the state variable `_request_queued` so it’s only called once.

Just like the server, `_write(`) calls `socket.send()` if there’s data in the send buffer.

The notable difference in the client’s version of `write()` is the last check to see if the request has been queued. This will be explained more in the section Client Main Script, but the reason for this is to tell `selector.select()` to stop monitoring the socket for write events. If the request has been queued and the send buffer is empty, then we’re done writing and we’re only interested in read events. There’s no reason to be notified that the socket is writable.

I’ll wrap up this section by leaving you with one thought. The main purpose of this section was to explain that `selector.select()` is calling into the `Message` class via the method `process_events()` and to describe how state is managed.

This is important because `process_events()` will be called many times over the life of the connection. Therefore, make sure that any methods that should only be called once are either checking a state variable themselves, or the state variable set by the method is checked by the caller.


#### Server Main Script

In the server’s main script `app-server.py`, arguments are read from the command line that specify the interface and port to listen on:

>```
$ ./app-server.py
usage: ./app-server.py <host> <port>
```

For example, to listen on the loopback interface on port 65432, enter:

>```
$ ./app-server.py 127.0.0.1 65432
listening on ('127.0.0.1', 65432)
```

Use an empty string for `<host>` to listen on all interfaces.

After creating the socket, a call is made to `socket.setsockopt()` with the option `socket.SO_REUSEADDR:`

>```# Avoid bind() exception: OSError: [Errno 48] Address already in use
lsock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
```

Setting this socket option avoids the error `Address already in use`. You’ll see this when starting the server and a previously used TCP socket on the same port has connections in the `TIME_WAIT` state.

For example, if the server actively closed a connection, it will remain in the `TIME_WAIT` state for two minutes or more, depending on the operating system. If you try to start the server again before the `TIME_WAIT` state expires, you’ll get an `OSError exception` of `Address already in use`. This is a safeguard to make sure that any delayed packets in the network aren’t delivered to the wrong application.

The event loop catches any errors so the server can stay up and continue to run:

>```
while True:
    events = sel.select(timeout=None)
    for key, mask in events:
        if key.data is None:
            accept_wrapper(key.fileobj)
        else:
            message = key.data
            try:
                message.process_events(mask)
            except Exception:
                print('main: error: exception for',
                      f'{message.addr}:\n{traceback.format_exc()}')
                message.close()
```

When a client connection is accepted, a Message object is created:

>```
def accept_wrapper(sock):
    conn, addr = sock.accept()  # Should be ready to read
    print('accepted connection from', addr)
    conn.setblocking(False)
    message = libserver.Message(sel, conn, addr)
    sel.register(conn, selectors.EVENT_READ, data=message)
```

The `Message` object is associated with the socket in the call to `sel.register()` and is initially set to be monitored for read events only. Once the request has been read, we’ll modify it to listen for write events only.

An advantage of taking this approach in the server is that in most cases, when a socket is healthy and there are no network issues, it will always be writable.

If we told `sel.register()` to also monitor EVENT_WRITE, the event loop would immediately wakeup and notify us that this is the case. However, at this point, there’s no reason to wake up and call `send()` on the socket. There’s no response to send since a request hasn’t been processed yet. This would consume and waste valuable CPU cycles.

#### Server Message Class

In the section above labelled *Message Entry Point*, we looked at how the Message object was called into action when socket events were ready via `process_events()`. Now let’s look at what happens as data is read on the socket and a component, or piece, of the message is ready to be processed by the server.

The server’s message class is in `libserver.py`. The source code is given later in this lab.

The methods appear in the class in the order in which processing takes place for a message.

When the server has read at least 2 bytes, the fixed-length header can be processed:

>```
def process_protoheader(self):
    hdrlen = 2
    if len(self._recv_buffer) >= hdrlen:
        self._jsonheader_len = struct.unpack('>H',
                                             self._recv_buffer[:hdrlen])[0]
        self._recv_buffer = self._recv_buffer[hdrlen:]
```

The fixed-length header is a 2-byte integer in network (big-endian) byte order that contains the length of the JSON header. [`struct.unpack()`](https://docs.python.org/3/library/struct.html) is used to read the value, decode it, and store it in `self._jsonheader_len`. After processing the piece of the message it’s responsible for, `process_protoheader()` removes it from the receive buffer.

>**Note:** if you don't understand `struct.pack()` and `struct.unpack()`, click on the link above.  These routines are used to interpret bytes as packed binary data.  They can easily handle conversion to big or little endian, as well as network byte order.  We will discuss another method to do conversions (`hton, ntoh`) later in this lab.

Just like the fixed-length header, when there’s enough data in the receive buffer to contain the JSON header, it can be processed as well:

>```
def process_jsonheader(self):
    hdrlen = self._jsonheader_len
    if len(self._recv_buffer) >= hdrlen:
        self.jsonheader = self._json_decode(self._recv_buffer[:hdrlen],
                                            'utf-8')
        self._recv_buffer = self._recv_buffer[hdrlen:]
        for reqhdr in ('byteorder', 'content-length', 'content-type',
                       'content-encoding'):
            if reqhdr not in self.jsonheader:
                raise ValueError(f'Missing required header "{reqhdr}".')
```

The method `self._json_decode()` is called to decode and deserialize the JSON header into a dictionary. Since the JSON header is defined as Unicode with a UTF-8 encoding, utf-8 is hardcoded in the call. The result is saved to `self.jsonheader`. After processing the piece of the message it’s responsible for, `process_jsonheader()` removes it from the receive buffer.

Next is the actual content, or payload, of the message. It’s described by the JSON header in `self.jsonheader`. When `content-length` bytes are available in the receive buffer, the request can be processed:

>```
def process_request(self):
    content_len = self.jsonheader['content-length']
    if not len(self._recv_buffer) >= content_len:
        return
    data = self._recv_buffer[:content_len]
    self._recv_buffer = self._recv_buffer[content_len:]
    if self.jsonheader['content-type'] == 'text/json':
        encoding = self.jsonheader['content-encoding']
        self.request = self._json_decode(data, encoding)
        print('received request', repr(self.request), 'from', self.addr)
    else:
        # Binary or unknown content-type
        self.request = data
        print(f'received {self.jsonheader["content-type"]} request from',
              self.addr)
    # Set selector to listen for write events, we're done reading.
    self._set_selector_events_mask('w')
```

After saving the message content to the data variable, `process_request()` removes it from the receive buffer. Then, if the content type is JSON, it decodes and deserializes it. If it’s not, for this example application, it assumes it’s a binary request and simply prints the content type.

The last thing `process_request()` does is modify the selector to monitor write events only. In the server’s main script, `app-server.py`, the socket is initially set to monitor read events only. Now that the request has been fully processed, we’re no longer interested in reading.

A response can now be created and written to the socket. When the socket is writable, `create_response()` is called from `write()`:

>```
def create_response(self):
    if self.jsonheader['content-type'] == 'text/json':
        response = self._create_response_json_content()
    else:
        # Binary or unknown content-type
        response = self._create_response_binary_content()
    message = self._create_message(**response)
    self.response_created = True
    self._send_buffer += message
```

A response is created by calling other methods, depending on the content type. In this example application, a simple dictionary lookup is done for JSON requests when `action == 'search'`. You can define other methods for your own applications that get called here... which we will do later in this lab.

After creating the response message, the state variable `self.response_created` is set so `write()` doesn’t call `create_response()` again. Finally, the response is appended to the send buffer. This is seen by and sent via `_write()`.

One tricky bit to figure out was how to close the connection after the response is written. I put the call to `close` in the method `_write()`:

>```
def _write(self):
    if self._send_buffer:
        print('sending', repr(self._send_buffer), 'to', self.addr)
        try:
            # Should be ready to write
            sent = self.sock.send(self._send_buffer)
        except BlockingIOError:
            # Resource temporarily unavailable (errno EWOULDBLOCK)
            pass
        else:
            self._send_buffer = self._send_buffer[sent:]
            # Close when the buffer is drained. The response has been sent.
            if sent and not self._send_buffer:
                self.close()
```

Although it’s somewhat “hidden,” I think it’s an acceptable trade-off given that the Message class only handles one message per connection. After the response is written, there’s nothing left for the server to do. It’s completed its work.

#### Client Main Script

In the client’s main script `app-client.py`, arguments are read from the command line and used to create requests and start connections to the server:

>```
$ ./app-client.py
usage: ./app-client.py <host> <port> <action> <value>
```

Here’s an example:

>```
$ ./app-client.py 127.0.0.1 65432 search needle
```

After creating a dictionary representing the request from the command-line arguments, the host, port, and request dictionary are passed to `start_connection()`:

>```
def start_connection(host, port, request):
    addr = (host, port)
    print('starting connection to', addr)
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.setblocking(False)
    sock.connect_ex(addr)
    events = selectors.EVENT_READ | selectors.EVENT_WRITE
    message = libclient.Message(sel, sock, addr, request)
    sel.register(sock, events, data=message)
```

A socket is created for the server connection as well as a `Message` object using the `request` dictionary.

Like the server, the `Message` object is associated with the socket in the call to `sel.register()`. However, for the client, the socket is initially set to be monitored for both read and write events. Once the request has been written, we’ll modify it to listen for read events only.

This approach gives us the same advantage as the server: not wasting CPU cycles. After the request has been sent, we’re no longer interested in write events, so there’s no reason to wake up and process them.

#### Client Message Class

In the section above labelled *Message Entry Point*, we looked at how the message object was called into action when socket events were ready via `process_events()`. Now let’s look at what happens after data is read and written on the socket and a message is ready to be processed by the client.

The client’s message class is in `libclient.py`. The source code is give later in this lab.

The methods appear in the class in the order in which processing takes place for a message.

The first task for the client is to queue the request:

>```
def queue_request(self):
    content = self.request['content']
    content_type = self.request['type']
    content_encoding = self.request['encoding']
    if content_type == 'text/json':
        req = {
            'content_bytes': self._json_encode(content, content_encoding),
            'content_type': content_type,
            'content_encoding': content_encoding
        }
    else:
        req = {
            'content_bytes': content,
            'content_type': content_type,
            'content_encoding': content_encoding
        }
    message = self._create_message(**req)
    self._send_buffer += message
    self._request_queued = True
```

The dictionaries used to create the request, depending on what was passed on the command line, are in the client’s main script, `app-client.py`. The request dictionary is passed as an argument to the class when a `Message` object is created.

The request message is created and appended to the send buffer, which is then seen by and sent via `_write()`. The state variable `self._request_queued` is set so `queue_request()` isn’t called again.

After the request has been sent, the client waits for a response from the server.

The methods for reading and processing a message in the client are the same as the server. As response data is read from the socket, the process header methods are called: `process_protoheader()` and `process_jsonheader()`.

The difference is in the naming of the final process methods and the fact that they’re processing a response, not creating one: `process_response()`,` _process_response_json_content()`, and `_process_response_binary_content()`.

Last, but certainly not least, is the final call for `process_response()`:
>```
def process_response(self):
    # ...
    # Close when response has been processed
    self.close()
```


#### Message Class Wrapup

I’ll conclude the Message class discussion by mentioning a couple of things that are important to notice with a few of the supporting methods.

Any exceptions raised by the class are caught by the main script in its except clause:
>```
try:
    message.process_events(mask)
except Exception:
    print('main: error: exception for',
          f'{message.addr}:\n{traceback.format_exc()}')
    message.close()
```

Note the last line: `message.close()`.

This is a really important line, for more than one reason! Not only does it make sure that the socket is closed, but `message.close()` also removes the socket from being monitored by `select()`. This greatly simplifies the code in the class and reduces complexity. If there’s an exception or we explicitly raise one ourselves, we know `close()` will take care of the cleanup.

The methods `Message._read()` and `Message._write()` also contain something interesting:

>```
def _read(self):
    try:
        # Should be ready to read
        data = self.sock.recv(4096)
    except BlockingIOError:
        # Resource temporarily unavailable (errno EWOULDBLOCK)
        pass
    else:
        if data:
            self._recv_buffer += data
        else:
            raise RuntimeError('Peer closed.')
```

Note the except line: `except BlockingIOError:`.

`_write()` has one too. These lines are important because they catch a temporary error and skip over it using pass. The temporary error is when the socket would block, for example if it’s waiting on the network or the other end of the connection (its peer).

By catching and skipping over the exception with pass, `select()` will eventually call us again, and we’ll get another chance to read or write the data.

----

### Running the Application Client and Server

After all of this hard work, let’s have some fun and run some searches!

In these examples, I’ll run the server so it listens on all interfaces by passing an empty string for the host argument. This will allow me to run the client and connect from a virtual machine that’s on another network. It emulates a big-endian PowerPC machine.

First, let’s start the server:
>```
$ ./app-server.py '' 65432
listening on ('', 65432)
```

Now let’s run the client and enter a search. Let’s see if we can find him:

>```
$ ./app-client.py 10.0.1.1 65432 search morpheus
starting connection to ('10.0.1.1', 65432)
sending b'\x00d{"byteorder": "big", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 41}{"action": "search", "value": "morpheus"}' to ('10.0.1.1', 65432)
received response {'result': 'Follow the white rabbit. 🐰'} from ('10.0.1.1', 65432)
got result: Follow the white rabbit. 🐰
closing connection to ('10.0.1.1', 65432)
```

My terminal is running a shell that’s using a text encoding of Unicode (UTF-8), so the output above prints nicely with emojis.

Let’s see if we can find the puppies:

>```
$ ./app-client.py 10.0.1.1 65432 search 🐶
starting connection to ('10.0.1.1', 65432)
sending b'\x00d{"byteorder": "big", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 37}{"action": "search", "value": "\xf0\x9f\x90\xb6"}' to ('10.0.1.1', 65432)
received response {'result': '🐾 Playing ball! 🏐'} from ('10.0.1.1', 65432)
got result: 🐾 Playing ball! 🏐
closing connection to ('10.0.1.1', 65432)
```

Notice the byte string sent over the network for the request in the sending line. It’s easier to see if you look for the bytes printed in hex that represent the puppy emoji: \xf0\x9f\x90\xb6. I was able to enter the emoji for the search since my terminal is using Unicode with the encoding UTF-8.

This demonstrates that we’re sending raw bytes over the network and they need to be decoded by the receiver to be interpreted correctly. This is why we went to all of the trouble to create a header that contains the content type and encoding.

Here’s the server output from both client connections above:

>```
accepted connection from ('10.0.2.2', 55340)
received request {'action': 'search', 'value': 'morpheus'} from ('10.0.2.2', 55340)
sending b'\x00g{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 43}{"result": "Follow the white rabbit. \xf0\x9f\x90\xb0"}' to ('10.0.2.2', 55340)
closing connection to ('10.0.2.2', 55340)
```
 
>```
accepted connection from ('10.0.2.2', 55338)
received request {'action': 'search', 'value': '🐶'} from ('10.0.2.2', 55338)
sending b'\x00g{"byteorder": "little", "content-type": "text/json", "content-encoding": "utf-8", "content-length": 37}{"result": "\xf0\x9f\x90\xbe Playing ball! \xf0\x9f\x8f\x90"}' to ('10.0.2.2', 55338)
closing connection to ('10.0.2.2', 55338)
```

Look at the sending line to see the bytes that were written to the client’s socket. This is the server’s response message.

You can also test sending binary requests to the server if the action argument is anything other than search:
>```
$ ./app-client.py 10.0.1.1 65432 binary 😃
starting connection to ('10.0.1.1', 65432)
sending b'\x00|{"byteorder": "big", "content-type": "binary/custom-client-binary-type", "content-encoding": "binary", "content-length": 10}binary\xf0\x9f\x98\x83' to ('10.0.1.1', 65432)
received binary/custom-server-binary-type response from ('10.0.1.1', 65432)
got response: b'First 10 bytes of request: binary\xf0\x9f\x98\x83'
closing connection to ('10.0.1.1', 65432)
```


Since the request’s content-type is not text/json, the server treats it as a custom binary type and doesn’t perform JSON decoding. It simply prints the content-type and returns the first 10 bytes to the client:

>```
$ ./app-server.py '' 65432
listening on ('', 65432)
accepted connection from ('10.0.2.2', 55320)
received binary/custom-client-binary-type request from ('10.0.2.2', 55320)
sending b'\x00\x7f{"byteorder": "little", "content-type": "binary/custom-server-binary-type", "content-encoding": "binary", "content-length": 37}First 10 bytes of request: binary\xf0\x9f\x98\x83' to ('10.0.2.2', 55320)
closing connection to ('10.0.2.2', 55320)
```


### Reference and Troubleshooting

The original web page tutorial includes a reference section that presents external links on socket programming, lists of error codes, etc.  There is also a troubleshooting section that talks about ping, wireshark, etc.  The links to these sections are given below. 

[Reference](https://realpython.com/python-sockets/#reference)

[Troubleshooting Tools](https://realpython.com/python-sockets/#troubleshooting)

----

### Your Assignment:  *Modify the application to add new commands*

Finally...  you get to do something besides read!

Your goal is to add two commands, `double` and `negate`, to the application.  

In order to do this you will use binary format for data interchange between the client and the server,rather than JSON format as used in the `search` command.  The reason for this is to teach you the method that many applications use to intercommunicate.  Binary format is much more efficient on the wire than sending JSON.  JSON, of course, is easier for humans to read.  Binary format is not.  That is why wireshark has so many plugins to interpret the application level interface (e.g. DNS interpretation).  

Here are the steps you should implement; you can do this with about a twenty lines of code:

1. Modify app-client.py to check for the 2 new commands.  If so, pack a 10 byte variable using `struct.pack()` to contain the 6 byte string with the command name and a 4-byte integer in network order (hint: the format string should be ">6si").  Set the JSON header to indicate binary data just like the default case.  The content should be your packed variable.
2. Modify the libserver.py routine that processes binary data to check for the existence of either command. If so, unpack the 4 byte integer (don't forget byte order!  hint: format string ">i").  Do the appropriate math on the value.  Then pack a new variable with a 6 byte string "result" followed by the 4 byte integer result.  Send it as binary data back to the client.  
3. Modify libclient.py in the routine that checks for binary data.  If the byte array contains the word `result`, print the binary string then unpack the integer result and print a line containing "result: xxx" where xxx is your calculated return value.

Some things to watch out for:

* `struct.unpack` returns a tuple.  You only want to print the first element of the tuple.
* binary data is a python byte array, not a string.  You will have to convert the command string to a byte array using mystring.encode().  You will also have to do compare operations using something like b"negate" and b"double".


#### Sample Output

You can try various examples, here are some sample output from my experiments:

>```
python app-client.py 'localhost' 9000 double 7654321
starting connection to ('localhost', 9000)
sending b'\x00\x7f{"byteorder": "little", "content-type": "binary/custom-client-binary-type", "content-encoding": "binary", "content-length": 10}double\x00t\xcb\xb1' to ('localhost', 9000)
received binary/custom-server-binary-type response from ('localhost', 9000)
got response: b'result\x00\xe9\x97b'
result:  15308642
closing connection to ('localhost', 9000)
```
>```
python app-client.py 'localhost' 9000 negate 7654321
starting connection to ('localhost', 9000)
sending b'\x00\x7f{"byteorder": "little", "content-type": "binary/custom-client-binary-type", "content-encoding": "binary", "content-length": 10}negate\x00t\xcb\xb1' to ('localhost', 9000)
received binary/custom-server-binary-type response from ('localhost', 9000)
got response: b'result\xff\x8b4O'
result:  -7654321
closing connection to ('localhost', 9000)
```
>```
python app-client.py 'localhost' 9000 negate -300
starting connection to ('localhost', 9000)
sending b'\x00\x7f{"byteorder": "little", "content-type": "binary/custom-client-binary-type", "content-encoding": "binary", "content-length": 10}negate\xff\xff\xfe\xd4' to ('localhost', 9000)
received binary/custom-server-binary-type response from ('localhost', 9000)
got response: b'result\x00\x00\x01,'
result:  300
closing connection to ('localhost', 9000)
```

If you are getting strange numbers returned from the server, you should check to ensure you have correctly performed host to network and network to host packing and unpacking.

#### What to turn in

Create a tarball containing your four python code files as well as a text file of sample output (use different numbers that the ones I did so we can ensure you didn't just copy and paste my results).  Submit this to CANVAS.